# Airqdata: analysis tools for air quality data

Notebook version with output and functional links: [https://nbviewer.jupyter.org/gist/dr-1/450c275b1ad2cbf88e9c4325c5d032bc](https://nbviewer.jupyter.org/gist/dr-1/450c275b1ad2cbf88e9c4325c5d032bc)

**Table of contents**  
[InfluencAir / Luftdaten.info resources](#InfluencAir-/-Luftdaten.info-resources)  
[Irceline.be resources](#Irceline.be-resources)  
[Combining the sources](#Combining-the-sources)

In [ ]:
# Prepare notebook to render plots
%matplotlib inline

In [ ]:
import pandas as pd

# Import the analysis package to make it available in the notebook
import airqdata

# More convenient in some cases: import individual names from the package to avoid having
# to prepend them with "airqdata."
from airqdata import (compare_nearest_irceline_sensors, compare_sensor_data, describe,
                      influencair, irceline, luftdaten)

# Limit output length for readability
pd.set_option("display.max_rows", 10)

## InfluencAir / Luftdaten.info resources

### Download list of sensors from InfluencAir's Google Sheet

In [ ]:
influencair.Metadata(refresh_cache=True)
sensor_info = influencair.Metadata.sensors
sensor_info.head(4)

In [ ]:
len(sensor_info)

### Create a Sensor object and get the sensor's metadata and current measurements

In [ ]:
# PM sensor in Anderlecht
demo_sensor_id = "3803"
demo_sensor = influencair.Sensor(demo_sensor_id, refresh_cache=True)

In [ ]:
demo_sensor.influencair_metadata

In [ ]:
demo_sensor.luftdaten_metadata

In [ ]:
demo_sensor.luftdaten_metadata_url

In [ ]:
# Luftdaten.info provides current measurements along with the sensor metadata.
demo_sensor.current_measurements

### Open data history graphs of the sensor in a browser

In [ ]:
demo_sensor.open_madavi_graphs()

### Retrieve data history
Data are retrieved from cache or server and then cleaned.

In [ ]:
demo_sensor.get_measurements(start_date="2018-11-04",
                             end_date="2018-11-08")

Now we can analyze the measurement data.
### Inspect, summarize and plot data

In [ ]:
demo_sensor.measurements

In [ ]:
describe(demo_sensor.measurements)

In [ ]:
demo_sensor.plot_measurements()

### Inspect, summarize and plot hourly means

In [ ]:
demo_sensor.get_hourly_means()

In [ ]:
describe(demo_sensor.get_hourly_means())

In [ ]:
demo_sensor.plot_hourly_means()

### Check distribution of sample intervals
Time series analyses tend to be easier with regularly spaced intervals. How regular are ours? Ideally all data points will be in the same interval group.

In [ ]:
demo_sensor.intervals.head(10)

### List sensors near a given location

Defaults to searching within an 8 kilometer radius around the center of Brussels

In [ ]:
luftdaten.search_proximity()

Using different location parameters

In [ ]:
luftdaten.search_proximity(lat=51.22, lon=4.41, radius=5)  # Antwerp

In [ ]:
(near_sensors,
 hourly_means) = luftdaten.evaluate_near_sensors(start_date="2018-11-10",
                                                 end_date="2018-11-13",
                                                 radius=1,
                                                 quiet=True)

In [ ]:
hourly_means

## Irceline.be resources

### Get IRCELINE metadata
IRCELINE provides information about
- the phenomena it measures
- the stations where those phenomena are measured
- the sensors that measure them (represented as time series)

In [ ]:
irceline.Metadata()

In [ ]:
irceline.Metadata.phenomena

In [ ]:
irceline.Metadata.stations

In [ ]:
irceline.Metadata.get_stations_by_name("bru")

In [ ]:
irceline.Metadata.time_series

### How many stations measure a given phenomenon?

In [ ]:
irceline.Metadata.time_series["phenomenon"].value_counts()

### How many phenomena does a given station measure?

In [ ]:
irceline.Metadata.time_series["station_label"].value_counts().head()

In [ ]:
pd.set_option("display.max_rows", 6)

### Where is a given phenomenon measured?

In [ ]:
irceline.Metadata.query_time_series(phenomenon="ethylbenzene")

### Where is PM2.5 measured?

In [ ]:
irceline.Metadata.get_pm25_time_series()

### Where is PM10 measured?

In [ ]:
irceline.Metadata.get_pm10_time_series()

### What are the closest locations to Etterbeek where IRCELINE measures NO₂?
Using a location in Etterbeek as a reference point: 50.837°N 4.39°E

In [ ]:
irceline.Metadata.query_time_series("nitrogen dioxide",
                                    lat_nearest=50.837,
                                    lon_nearest=4.39)

### What does the Uccle station measure?

In [ ]:
irceline.Metadata.list_station_time_series("ucc")

### List stations near a location
Defaults to coordinates and radius of Brussels

In [ ]:
irceline.Metadata.search_proximity(lat=50.9, lon=4.4, radius=3)

### Create a sensor object from a time series, retrieve its measurements and plot them

In [ ]:
irceline_demo_sensor = irceline.Sensor("6615")  # An NO₂ sensor in Ixelles

In [ ]:
irceline_demo_sensor.get_measurements(start_date="2018-11-03",
                                      end_date="2018-11-08")

In [ ]:
irceline_demo_sensor.measurements.head()

In [ ]:
irceline_demo_sensor.plot_measurements()

## Combining the sources

In [ ]:
pd.set_option("display.max_rows", 10)

### Which are the closest IRCELINE sensors to a given luftdaten.info sensor that measure the same phenomenon?

In [ ]:
nearest = irceline.find_nearest_sensors(demo_sensor, quiet=True)
nearest

### Compare data of a luftdaten.info sensor and the nearest IRCELINE sensors

In [ ]:
combined_data, plots = compare_nearest_irceline_sensors(demo_sensor,
                                                        start_date="2018-11-03",
                                                        end_date="2018-11-10",
                                                        quiet=True)

#### Correlation between the compared values

In [ ]:
combined_data.corr()

### Compare data from any sensors

In [ ]:
t_rh_sensor = luftdaten.Sensor("5562")  # Temperature and humidity sensor at Brussels Central Station
combined_data, plot = compare_sensor_data(sensors=[demo_sensor, t_rh_sensor, t_rh_sensor, irceline_demo_sensor],
                                          phenomena=["pm2.5", "temperature", "humidity", "Nitrogen dioxide"],
                                          start_date="2018-11-05",
                                          end_date="2018-11-10",
                                          hourly_means=True,
                                          quiet=True)

In [ ]:
combined_data.head()

#### Correlation between the compared values

In [ ]:
combined_data.corr()

## Export data for use in another environment

In [ ]:
# demo_sensor.measurements.to_csv("demo_sensor_data.csv")

## More advanced analysis
We can analyze the measurement data using Pandas' extensive capabilities.

### Get more data

In [ ]:
demo_sensor.get_measurements(start_date="2018-05-24",
                             end_date="2018-11-23",
                             quiet=True)
data = demo_sensor.measurements["pm2.5"]

# Convert index to local time
data.index = data.index.tz_convert("Europe/Brussels")

describe(data)

### Summarize measurements by day of the week

In [ ]:
# Produce a statistical summary of the data grouped by day of the week
grouping_variable = data.index.dayofweek
weekday_summary = (data
                   .groupby(grouping_variable)
                   .describe(percentiles=[0.01, 0.99]))

# Show day names instead of integers
import calendar
calendar.setfirstweekday(1)  # Start week on Monday
weekday_summary.index = [calendar.day_abbr[i]
                         for i in weekday_summary.index]
weekday_summary.index.name = "Day of the Week (Local Time)"

# Get spread values
yspread = [[(weekday_summary["mean"] - weekday_summary["1%"]),
            (weekday_summary["99%"] - weekday_summary["mean"])]]

# Plot
title = ("PM2.5 Concentration by Day of the Week\n"
         "Mean and 98% Range\n"
         + demo_sensor.label)
ax = (weekday_summary["mean"]
      .plot(kind="bar", ylim=(0, None), color="black", title=title,
            yerr=yspread, legend=True, figsize=(12, 8)))
ax.set(ylabel="Concentration in µg/m³")
ax.xaxis.grid(False);

In [ ]:
weekday_summary

### Summarize measurements by hour of the day

In [ ]:
# Analyze data
grouping_variable = data.index.hour
hour_summary = (data
                .groupby(grouping_variable)
                .describe(percentiles=[0.01, 0.99]))
hour_summary.index.name = "Hour of the Day (Local Time)"

# Get spread values
yspread = [[(hour_summary["mean"] - hour_summary["1%"]),
            (hour_summary["99%"] - hour_summary["mean"])]]

# Plot
title = ("PM2.5 Concentration by Hour of the Day\n"
         "Mean and 98% Range\n"
         + demo_sensor.label)
ax = (hour_summary["mean"]
      .plot(kind="bar", ylim=(0, None), color="black", title=title,
            yerr=yspread, legend=True, figsize=(12, 8)))
ax.set(ylabel="Concentration in µg/m³")
ax.xaxis.grid(False);

In [ ]:
hour_summary